In [ ]:
import mne
import scipy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import utils
import algo
import os
import random
import glob
import copy
import itertools
import pickle
from findpeaks import findpeaks
from tqdm import tqdm
from numpy import linalg as LA
from scipy.stats import zscore, pearsonr
from scipy.io import savemat, loadmat
from scipy import signal
%matplotlib widget

In [ ]:
subjects = ['AS', 'YY', 'CM', 'SUB1', 'IR', 'SUB2', 'YZ', 'WD', 'CC', 'CW', 'WS', 'VC','HV','JC','DV','CD','JV','KY','KB','SC']
fs = 30
with open('data/OneShot/features.pkl', 'rb') as f:
    features_list = pickle.load(f)
with open('data/OneShot/eeg_multisub.pkl', 'rb') as f:
    eeg_multisub_list = pickle.load(f)
with open('data/OneShot/eog_multisub.pkl', 'rb') as f:
    eog_multisub_list = pickle.load(f)
with open('data/OneShot/hf_multisub.pkl', 'rb') as f:
    hf_multisub_list = pickle.load(f)

In [ ]:
avgflow_list = [feats[:,20] for feats in features_list]
avgtempctr_list = [feats[:,21] for feats in features_list]
objflow_list = [feats[:,8] for feats in features_list]
objtempctr_list = [feats[:,17] for feats in features_list]

In [ ]:
T = sum([F.shape[0] for F in features_list])
times = np.array(range(T))/fs
n_sub = len(subjects)
T/fs/60

In [ ]:
def pip_ISC_nb_sub(method, eeg_multisub_list, feature_list, fs, nb_MC=20, fold=5, fold_val=10, trials=True, crs_val=True):
    subject_list = range(len(subjects))
    corr_mean = []
    tsc_mean = []
    corr_std = []
    tsc_std = []
    if crs_val:
        filename = 'tables/nb_sub/' + method + '_fold_' + str(fold) +  '_fold_val_' + str(fold_val) + '_MC_' + str(nb_MC) + '.npz'
    else:
        filename = 'tables/nb_sub/' + method + '_no_cross_val' + '_MC_' + str(nb_MC) + '.npz'
    for n_sub in range(5, len(subjects)+1):
        print('number of subjects: ', n_sub)
        corr_list = []
        tsc_list = []
        count = 0
        comb_list = list(itertools.combinations(subject_list, n_sub))
        random.shuffle(comb_list)
        for item in comb_list:
            count += 1
            if count > nb_MC:
                break
            eeg = [eeg[:,:,list(item)] for eeg in eeg_multisub_list]
            nested_datalist = [eeg, feature_list]
            Llist = [5, int(fs/2)]
            offsetlist = [2, 0]
            nested_update, _, _, _  = utils.get_val_set(nested_datalist, fold=fold, fold_val=fold_val, crs_val=crs_val)
            if method == 'GCCA':
                GCCA = algo.GeneralizedCCA(nested_update[0], fs, L=Llist[0], offset=offsetlist[0], fold=fold, n_components=10, signifi_level=False, message=False, trials=trials, crs_val=crs_val)
                _, corr_test, _, _, _, tsc_test, _, _, _, _ = GCCA.cross_val()
            elif method == 'SIGCCA':
                SI_GCCA = algo.StimulusInformedGCCA(nested_datalist, fs, Llist, offsetlist, fold=fold, n_components=10, signifi_level=False, message=False, trials=trials, crs_val=crs_val)
                _, corr_test, _, tsc_test, _, _, _, _, _ = SI_GCCA.cross_val()
            elif method == 'CorrCA':
                corr_CA = algo.CorrelatedComponentAnalysis(nested_update[0], fs, L=Llist[0], offset=offsetlist[0], fold=fold, n_components=10, signifi_level=False, message=False, trials=trials, crs_val=crs_val)
                _, corr_test, _, _,  _, tsc_test, _, _, _, _ = corr_CA.cross_val()
            elif method == 'SICorrCA':
                SI_CorrCA = algo.StimulusInformedCorrCA(nested_datalist, fs, Llist, offsetlist, fold=fold, n_components=10, signifi_level=False, message=False, trials=trials, crs_val=crs_val)
                _, corr_test, _, tsc_test, _, _, _, _, _ = SI_CorrCA.cross_val()
            else:
                raise ValueError('method not defined')
            corr_list.append(corr_test)
            tsc_list.append(tsc_test)
        corr_all = np.concatenate(tuple(corr_list), axis=0)
        tsc_all = np.concatenate(tuple(tsc_list), axis=0)
        corr_mean.append(np.mean(corr_all, axis=0))
        tsc_mean.append(np.mean(tsc_all))
        print('corr_mean: ', np.mean(corr_all, axis=0))
        corr_std.append(np.std(corr_all, axis=0))
        tsc_std.append(np.std(tsc_all))
        np.savez(filename, tsc_mean=tsc_mean, tsc_std=tsc_std, corr_mean=corr_mean, corr_std=corr_std)
    return corr_mean, tsc_mean, corr_std, tsc_std

In [ ]:
GCCA_corr_mean, _, GCCA_corr_std, _ = pip_ISC_nb_sub('GCCA', eeg_multisub_list, objflow_list, fs, nb_MC=50, fold_val=20, crs_val=False)

In [ ]:
SIGCCA_corr_mean, _, SIGCCA_corr_std, _ = pip_ISC_nb_sub('SIGCCA', eeg_multisub_list, objflow_list, fs, nb_MC=50, fold_val=20, crs_val=False)

In [ ]:
CorrCA_corr_mean, _, CorrCA_corr_std, _ = pip_ISC_nb_sub('CorrCA', eeg_multisub_list, objflow_list, fs, nb_MC=50, fold_val=20, crs_val=False)

In [ ]:
SICorrCA_corr_mean, _, SICorrCA_corr_std, _ = pip_ISC_nb_sub('SICorrCA', eeg_multisub_list, objflow_list, fs, nb_MC=50, fold_val=20, crs_val=False)

In [ ]:
def pip_ISC_am_data(method, eeg_multisub_list, feature_list, fs, nb_MC=20, fold=5, fold_val=10, trials=True, crs_val=True):
    eeg_multisub_concat = np.concatenate(tuple(eeg_multisub_list), axis=0)
    feature_concat = np.concatenate(tuple(feature_list), axis=0)
    nb_samples = eeg_multisub_concat.shape[0]
    len_list = np.linspace(10, 55, 10).astype(int)
    corr_mean = []
    tsc_mean = []
    corr_std = []
    tsc_std = []
    if crs_val:
        filename = 'tables/am_data/' + method + '_fold_' + str(fold) +  '_fold_val_' + str(fold_val) + '_MC_' + str(nb_MC) + '.npz'
    else:
        filename = 'tables/am_data/' + method + '_no_cross_val' + '_MC_' + str(nb_MC) + '.npz'
    for length in len_list:
        print('amount of data (min): ', length)
        len_block = int(length*fs*60/(1-1/fold_val))
        idx_end = nb_samples-len_block
        corr_list = []
        tsc_list = []
        for count in range(nb_MC):
            start_point = random.randint(0, idx_end)
            eeg = [eeg_multisub_concat[start_point:start_point+len_block,:,:]]
            feature = [feature_concat[start_point:start_point+len_block]]
            nested_datalist = [eeg, feature]
            Llist = [5, int(fs/2)]
            offsetlist = [2, 0]
            nested_update, _, _, _  = utils.get_val_set(nested_datalist, fold=fold, fold_val=fold_val, crs_val=crs_val)
            # assert nested_update[0][0].shape[0] == int(length*fs*60)
            if method == 'GCCA':
                GCCA = algo.GeneralizedCCA(nested_update[0], fs, L=Llist[0], offset=offsetlist[0], fold=fold, n_components=10, signifi_level=False, message=False, trials=trials, crs_val=crs_val)
                _, corr_test, _, _, _, tsc_test, _, _, _, _ = GCCA.cross_val()
            elif method == 'SIGCCA':
                SI_GCCA = algo.StimulusInformedGCCA(nested_datalist, fs, Llist, offsetlist, fold=fold, n_components=10, signifi_level=False, message=False, trials=trials, crs_val=crs_val)
                _, corr_test, _, tsc_test, _, _, _, _, _ = SI_GCCA.cross_val()
            elif method == 'CorrCA':
                corr_CA = algo.CorrelatedComponentAnalysis(nested_update[0], fs, L=Llist[0], offset=offsetlist[0], fold=fold, n_components=10, signifi_level=False, message=False, trials=trials, crs_val=crs_val)
                _, corr_test, _, _,  _, tsc_test, _, _, _, _ = corr_CA.cross_val()
            elif method == 'SICorrCA':
                SI_CorrCA = algo.StimulusInformedCorrCA(nested_datalist, fs, Llist, offsetlist, fold=fold, n_components=10, signifi_level=False, message=False, trials=trials, crs_val=crs_val)
                _, corr_test, _, tsc_test, _, _, _, _, _ = SI_CorrCA.cross_val()
            else:
                raise ValueError('method not defined')
            corr_list.append(corr_test)
            tsc_list.append(tsc_test)
        corr_all = np.concatenate(tuple(corr_list), axis=0)
        tsc_all = np.concatenate(tuple(tsc_list), axis=0)
        corr_mean.append(np.mean(corr_all, axis=0))
        tsc_mean.append(np.mean(tsc_all))
        print('corr_mean: ', np.mean(corr_all, axis=0))
        corr_std.append(np.std(corr_all, axis=0))
        tsc_std.append(np.std(tsc_all))
        np.savez(filename, tsc_mean=tsc_mean, tsc_std=tsc_std, corr_mean=corr_mean, corr_std=corr_std)
    return corr_mean, tsc_mean, corr_std, tsc_std

In [ ]:
GCCA_corr_mean, _, GCCA_corr_std, _ = pip_ISC_am_data('GCCA', eeg_multisub_list, objflow_list, fs, nb_MC=50, fold_val=20, crs_val=False)

In [ ]:
SIGCCA_corr_mean, _, SIGCCA_corr_std, _  = pip_ISC_am_data('SIGCCA', eeg_multisub_list, objflow_list, fs, nb_MC=50, fold_val=20, crs_val=False)

In [ ]:
CorrCA_corr_mean, _, CorrCA_corr_std, _ = pip_ISC_am_data('CorrCA', eeg_multisub_list, objflow_list, fs, nb_MC=50, fold_val=20, crs_val=False)

In [ ]:
SICorrCA_corr_mean, _, SICorrCA_corr_std, _ = pip_ISC_am_data('SICorrCA', eeg_multisub_list, objflow_list, fs, nb_MC=50, fold_val=20, crs_val=False)